# Exploração de Dados das Bombas ITU - 415 e ITU - 844

#### Impotação das Bibliotecas

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

## ITU - 415

O primeiro passo vai ser verificar os dados dessa máquina e concatenar o 4 datasets em um único csv

In [ ]:
import os
print(os.listdir("../data"))

In [ ]:
arquivos = [
    "../data/full_history_ITU-415_2025-06-01_a_2025-06-17.csv",
    "../data/full_history_ITU-415_2025-06-17_a_2025-06-29.csv",
    "../data/full_history_ITU-415_2025-06-29_a_2025-06-31.csv",
    "../data/full_history_ITU-415_2025-07-01_a_2025-07-17 (1).csv"
]

total = 0
for f in arquivos:
    nlinhas = pd.read_csv(f).shape[0]
    print(f"{f} → {nlinhas} linhas")
    total += nlinhas

print(f"\nTotal de linhas somadas: {total}")

#### Concatenação

In [ ]:
df_415 = pd.concat([pd.read_csv(f) for f in arquivos], ignore_index=True)

print(df_415.shape)
df_415.head()

In [ ]:
print("Sensores únicos:", df_415['resource'].nunique())
print(df_415['resource'].unique())  # lista todos, na ordem em que aparecem no DataFrame
print("Período:", df_415['timestamp'].min(), "→", df_415['timestamp'].max())


#### Interpretação dos sensores (com base nos comentários e discussões com o parceiro):

1. Eng_RPM: Rotação do motor
2. Recalque: Sensor da pressão de descarga (saída) da bomba centrífuga
3. Oil_L: Nível de Óleo do Motor
4. Cool_T: Temperatura do líquido de arrefecimento
5. Fuel_L: Nível de combustível no tanque do motor
6. Char_V: Tensão no alternador
7. Vibracao: Vibração da centrífuga (não usada)
8. FlexAnalogue1_1:
9. FlexAnalogue1_2:
10. FlexAnalogue2_1:
11. FlexAnalogue2_2:
12. FlexAnalogue3_1:
13. FlexAnalogue3_2:
14. FlexAnalogue4_1:
15. FlexAnalogue4_2:
16. FlexAnalogue6_1: Sensor de sucção (entrada) da bomba centrífuga (de acordo com o TAPI)
17. FlexAnalogue6_2: Sensor de sucção (entrada) da bomba centrífuga (de acordo com o TAPI)
18. FlexAnalogue7_1: Sensor de sucção (entrada) da bomba centrífuga (de acordo com o TAPI)
19. FlexAnalogue7_2: 
20. FlexAnalogue8_1:
21. FlexAnalogue8_2:
22. FlexAnalogue9_1:
23. FlexAnalogue9_2:
24. FlexAnalogue10_1:
25. FlexAnalogue10_2:
26. FlexAnalogue11_1:
27. FlexAnalogue11_2:
28. FlexAnalogue12_1:
29. FlexAnalogue12_2:
30. Stop: Indicação de máquina parada no painel de controle (0 operando; 1 parada)
31. Man: Modo manual do painel de controle
32. Auto: Modo automático do painel de controle
33. Bat_V: Tensão na Bateria
34. Fuel_Con: Consumo de combustível do motor (não usada)
35. Oil_P: Pressão de Óleo no Motor
36. EngRT_H: Tempo de operação da máquina
37. Starts_N: Quantidade de vezes em que a máquina foi ligada
38. lon:

#### Tentando entender o flexanalogue (e entender o pq nao tem sucção)

In [ ]:
# filtra apenas os sensores FlexAnalogue
mask_flex = df_415['resource'].str.contains('FlexAnalogue', na=False)
df_flex = df_415[mask_flex].copy()
df_flex['value'] = pd.to_numeric(df_flex['value'], errors='coerce')
df_flex.loc[df_flex['value'] > 1e9, 'value'] = np.nan

# resumo por sensor
resumo = df_flex.groupby('resource')['value'].agg(
    total_registros='count',
    valores_validos=lambda s: s.notna().sum(),
    pct_nulos=lambda s: s.isna().mean()*100,
    media='mean',
    std='std',
    minimo='min',
    maximo='max'
).sort_values(by='pct_nulos')

resumo


In [ ]:
# mantém só FlexAnalogue4_1 e FlexAnalogue4_2
df_415 = df_415[
    ~df_415['resource'].str.contains('FlexAnalogue', na=False) | 
    df_415['resource'].isin(['FlexAnalogue4_1', 'FlexAnalogue4_2'])
]


In [ ]:
print("Sensores únicos:", df_415['resource'].nunique())
print(df_415['resource'].unique())  # lista todos, na ordem em que aparecem no DataFrame


In [ ]:
print(df_wide[['FlexAnalogue4_1','FlexAnalogue4_2']].info())

In [ ]:
print(df_wide[['FlexAnalogue4_1','FlexAnalogue4_2']].describe())


In [ ]:
print(df_wide[['FlexAnalogue4_1','FlexAnalogue4_2']].notna().sum())


In [ ]:
df_wide = df_415.pivot_table(index='timestamp', columns='resource', values='value', aggfunc='last')

sns.scatterplot(x=df_wide['FlexAnalogue4_1'], y=df_wide['FlexAnalogue4_2'])
plt.title("Comparação FlexAnalogue4_1 vs FlexAnalogue4_2")
plt.show()

corr = df_wide[['FlexAnalogue4_1','FlexAnalogue4_2']].corr()
print(corr)


In [ ]:
print(df_415.isna().sum())


#### Converter a coluna 'timestamp' em datetime

In [ ]:
df_415['timestamp'] = pd.to_datetime(df_415['timestamp'], errors='coerce')

inicio = df_415['timestamp'].min()
fim = df_415['timestamp'].max()
duracao = fim - inicio

print("Início:", inicio)
print("Fim:", fim)
print("Duração total:", duracao)
print("Dias totais:", duracao.days)

In [ ]:
# pivot wide
df_wide = df_415.pivot_table(index="timestamp", columns="resource", values="value", aggfunc="last")

In [ ]:
df_wide.info()

In [ ]:
# testar 3 granularidades
for j in ['1S','15S','1min']:
    linhas = len(df_wide.resample(j).mean())
    print(f"{j}: {linhas} linhas")

In [ ]:
print(df_wide.shape)

## ITU-844

In [ ]:
import os
print(os.listdir("../data"))

In [ ]:
file_path = "../data/Cópia de alertas_ITU-844_2025-06-22_a_2025-07-21.csv"
df_844 = pd.read_csv(file_path)

print(df_844.head())

In [ ]:
df_844.info()
df_844.describe(include="all")  # estatísticas descritivas de todas as colunas


In [ ]:
df_844.isna().sum()
df_844.duplicated().sum()

In [ ]:
print(df_844.columns)


In [ ]:
print(df_415.columns)

In [ ]:
print("Sensores únicos:", df_844['resource'].nunique())
print(df_844['resource'].unique())  # lista todos, na ordem em que aparecem no DataFrame
print("Período:", df_844['timestamp'].min(), "→", df_844['timestamp'].max())

In [ ]:
df_415.info()